<a href="https://colab.research.google.com/github/EEarth1270/houseprice-prediction/blob/randomforest/HousePriceRandomForesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from scipy.stats import iqr
from pandas.api.types import CategoricalDtype 
import numpy as np

In [35]:
train_dataset = pd.read_csv('/content/drive/My Drive/Datasci/train.csv')

# Pre-processing

In [ ]:
corrMatrix = train_dataset.corr()
print(corrMatrix['SalePrice'].sort_values(ascending=False))

In [36]:
print(train_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [28]:
cols_with_no_nans = []
for col in train_dataset.columns:
  if not train_dataset[col].isnull().any():
    if col != 'SalePrice':
      cols_with_no_nans.append(col)

In [29]:
print(cols_with_no_nans)

['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [30]:
salePrice = train_dataset['SalePrice']

In [38]:
train_dataset= train_dataset[cols_with_no_nans]

In [39]:
test_dataset = pd.read_csv('/content/drive/My Drive/Datasci/test.csv')

In [40]:
test_dataset= test_dataset[cols_with_no_nans]

In [42]:
dataset =  train_dataset.append(test_dataset)

In [8]:
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df

In [43]:
dataset = oneHotEncode(dataset,cols_with_no_nans)

In [44]:
print(dataset)

        Id  MSSubClass  ...  SaleCondition_Normal  SaleCondition_Partial
0        1          60  ...                     1                      0
1        2          20  ...                     1                      0
2        3          60  ...                     1                      0
3        4          70  ...                     0                      0
4        5          60  ...                     1                      0
...    ...         ...  ...                   ...                    ...
1454  2915         160  ...                     1                      0
1455  2916         160  ...                     0                      0
1456  2917          20  ...                     0                      0
1457  2918          85  ...                     1                      0
1458  2919          60  ...                     1                      0

[2919 rows x 216 columns]


In [45]:
train_dataset = dataset[:1460]
test_dataset = dataset[1460:]

In [55]:
print(test_dataset)

        Id  MSSubClass  ...  SaleCondition_Normal  SaleCondition_Partial
0     1461          20  ...                     1                      0
1     1462          20  ...                     1                      0
2     1463          60  ...                     1                      0
3     1464          60  ...                     1                      0
4     1465         120  ...                     1                      0
...    ...         ...  ...                   ...                    ...
1454  2915         160  ...                     1                      0
1455  2916         160  ...                     0                      0
1456  2917          20  ...                     0                      0
1457  2918          85  ...                     1                      0
1458  2919          60  ...                     1                      0

[1459 rows x 216 columns]


In [32]:
# train_dataset['MSSubClass'] = train_dataset['MSSubClass'].astype(CategoricalDtype([20,30,40,45,50,60,70,75,80,85,90,120,150,160,180,190]))
train_dataset['OverallQual'] = train_dataset['OverallQual'].astype(CategoricalDtype([1,2,3,4,5,6,7,8,9,10]))
# train_dataset['OverallCond'] = train_dataset['OverallCond'].astype(CategoricalDtype([1,2,3,4,5,6,7,8,9,10]))

In [ ]:
print(train_dataset)

        Id  MSSubClass  ...  SaleCondition_Normal  SaleCondition_Partial
0        1          60  ...                     1                      0
1        2          20  ...                     1                      0
2        3          60  ...                     1                      0
3        4          70  ...                     0                      0
4        5          60  ...                     1                      0
...    ...         ...  ...                   ...                    ...
1455  1456          60  ...                     1                      0
1456  1457          20  ...                     1                      0
1457  1458          70  ...                     1                      0
1458  1459          20  ...                     1                      0
1459  1460          20  ...                     1                      0

[1460 rows x 236 columns]


In [11]:
train_dataset = pd.concat([train_dataset,salePrice],axis=1)

In [ ]:
train_dataset.dropna(inplace=True)

In [12]:
print(train_dataset)

        Id  MSSubClass  ...  SaleCondition_Partial  SalePrice
0        1          60  ...                      0     208500
1        2          20  ...                      0     181500
2        3          60  ...                      0     223500
3        4          70  ...                      0     140000
4        5          60  ...                      0     250000
...    ...         ...  ...                    ...        ...
1455  1456          60  ...                      0     175000
1456  1457          20  ...                      0     210000
1457  1458          70  ...                      0     266500
1458  1459          20  ...                      0     142125
1459  1460          20  ...                      0     147500

[1460 rows x 217 columns]


In [56]:
print(train_dataset.iloc[:,1:-1])

      MSSubClass  LotArea  ...  SaleCondition_Family  SaleCondition_Normal
0             60     8450  ...                     0                     1
1             20     9600  ...                     0                     1
2             60    11250  ...                     0                     1
3             70     9550  ...                     0                     0
4             60    14260  ...                     0                     1
...          ...      ...  ...                   ...                   ...
1455          60     7917  ...                     0                     1
1456          20    13175  ...                     0                     1
1457          70     9042  ...                     0                     1
1458          20     9717  ...                     0                     1
1459          20     9937  ...                     0                     1

[1460 rows x 214 columns]


In [48]:
train_dataset.columns

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=216)

# Training

In [105]:
reg = RandomForestRegressor(n_estimators=2500,max_features=0.33,ccp_alpha=0.05,n_jobs=-1)

In [106]:
reg.fit(train_dataset.iloc[:,1:],salePrice)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.05, criterion='mse',
                      max_depth=None, max_features=0.33, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2500, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [53]:
test_dataset = test_dataset.fillna(0)


In [20]:
print(test_dataset[cols_with_no_nans].isna().sum())

Id               0
MSSubClass       0
MSZoning         0
LotArea          0
Street           0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 61, dtype: int64


In [102]:
print(test_dataset.iloc[:,1:])

      MSSubClass  LotArea  ...  SaleCondition_Normal  SaleCondition_Partial
0             20    11622  ...                     1                      0
1             20    14267  ...                     1                      0
2             60    13830  ...                     1                      0
3             60     9978  ...                     1                      0
4            120     5005  ...                     1                      0
...          ...      ...  ...                   ...                    ...
1454         160     1936  ...                     1                      0
1455         160     1894  ...                     0                      0
1456          20    20000  ...                     0                      0
1457          85    10441  ...                     1                      0
1458          60     9627  ...                     1                      0

[1459 rows x 215 columns]


# Testing

In [107]:
result = reg.predict(test_dataset.iloc[:,1:])

In [108]:
print(result)

[126836.8632 156175.5488 177541.0528 ... 161118.7908 115683.3964
 229940.662 ]


In [109]:
ans=pd.DataFrame({'SalePrice':result})
submission = pd.concat([test_dataset['Id'],ans],axis=1)
submission.to_csv('/content/drive/My Drive/Datasci/outputrandom_forest25.csv', index=False)